# Figures

Create static figures from tau_map6 simulation.

## Notebook Configuration

### Imports

In [1]:
# import packages
import os
import warnings
from os import PathLike
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

from matplotlib import colormaps
from matplotlib.ticker import MaxNLocator
from matplotlib.animation import FuncAnimation

from rich import print
from tqdm import tqdm

In [2]:
# import modules
from configurations import DEF_VAR_LIST, DEF_REM_COL_LIST, CSTYLE_FILE_STR
from grouped_bar_plot import grouped_bar_plot
from processing import load_mat_dir

In [3]:
# define the data directory
data_dir = Path("data")

# configure the figure save directory
figure_dir = Path("figures", data_dir.parts[-1])
os.makedirs(figure_dir, exist_ok=True)

# set plotting style
plt.style.use(["default", "cstyle.mplstyle"])

### Data Import

In [4]:
# get the data dictionary from the data directory
data_dict = load_mat_dir(data_dir, DEF_VAR_LIST, DEF_REM_COL_LIST)

### Add Calculations

In [23]:
# add the final MT length to the sim_df
data_dict["sims"].keys()
length_vals = {}
for sim_name, sim_dict in data_dict["sims"].items():
    # extract the final length as the last entry in first col of mt_length
    length_vals[sim_name] = sim_dict["mt_length"][-1, 0]

# add the dictionary to the df
data_dict["df"]["final_length_units"] = data_dict["df"].index.map(length_vals)

# add final length column
data_dict["df"]["final_length"] = data_dict["df"]["final_length_units"]*data_dict["df"]["dx"]

In [6]:
# create an averages df grouped by tm_ratio
data_avg_df = data_dict["df"].groupby("tm_ratio").mean()

## Averages DF Plotting

### Binding Rate Ratio vs. MT Length

In [7]:
# define the grouped data and group labels
group_data = {"L": data_avg_df[""]}
group_labels = data_avg_df.index

# define the figure
fig, ax = plt.subplots()

# plot the grouped bar plot
fig, ax = grouped_bar_plot(fig, ax, group_data, group_labels, multiplier=1)

# add a margin via xmargin to the x axis
ax.margins(x=0.1)

# set the labels
ax.set_xlabel(r"Ratio of tau binding rate to map6 binding rate")
ax.set_ylabel(r"Microtubule length after 5 minutes [$\qty{}{\micro\meter}$]")
ax.set_title(r"Effect of binding ratio on microtubule growth")

# save the figure
fig.savefig(figure_dir / r"growth_effect")

KeyError: 'L'